# 0. Imports

## 0.1 Bibliotecas

In [1]:
from IPython.display import display, HTML

In [2]:
import sys

sys.path.insert(0, '../src/')

import joblib
from sklearn.pipeline              import Pipeline
from sklearn.preprocessing         import StandardScaler
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.imputation     import MeanMedianImputer
from feature_engine.wrappers       import SklearnTransformerWrapper
from sklearn.linear_model          import LogisticRegression

from utils.utils               import load_config_file
from data.dataload             import DataLoad
from data.datavalidation       import DataValidation
from data.datatranformation    import DataTransformation
# from data.datapreprocessing    import DataPreprocess
# from train.train               import TrainModel
from evaluation.classfier_eval import ModelEvaluation

## 0.2 Funções de Ajuda

## 0.3 Configurações do Jupyter

In [3]:
# Deixar o jupyper em widescreen
display(HTML("<style>.container { width:90% !important; }</style>"))

# Seta o máximo de colunas e linhas que o pandas vai exibir
# pd.set_option('display.max_columns', 20)
# pd.set_option('display.max_rows', 60)

# 1. DataLoad

In [4]:
dl = DataLoad()
df = dl.load_data('train_dataset_name')
df.head(1)

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0


# 2.Data Validation 

In [5]:
dv = DataValidation()
dv.run(dataframe=df)

2024-01-08 11:33:53 [info     ] Validação Concluída           


True

# 3. DataTransformation

In [6]:
dt = DataTransformation(df)
X_train, X_val, y_train, y_val = dt.train_test_spliting()

# 4. Experimentation

In [7]:
import mlflow
from mlflow.tracking import MlflowClient

In [8]:
mlflow.set_tracking_uri('http://127.0.0.1:5000/')
mlflow.set_experiment('prob_learng')

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1704470502137, experiment_id='1', last_update_time=1704470502137, lifecycle_stage='active', name='prob_learng', tags={}>

## 4.1 Select Best Model

In [9]:
current_experiment = dict(mlflow.get_experiment_by_name('prob_learng'))

In [10]:
experiment_id = current_experiment['experiment_id']

In [11]:
df_mlflow = mlflow.search_runs(filter_string='metrics.valid_roc_auc <1').sort_values('metrics.valid_roc_auc', ascending=False)

In [12]:
run_id = df_mlflow.loc[df_mlflow['metrics.valid_roc_auc'].idxmax(), 'run_id']

In [13]:
list(df_mlflow.columns)